In [645]:
import pandas as pd
import numpy as np

In [646]:
path = "../data/top_100_saas_companies_2025.csv"
df = pd.read_csv(path)

In [647]:
df = df.drop(columns=["Unnamed: 0"])
renamed_columns = {    "Company Name": "nom_entreprise",
            "Founded Year": "annee_creation",
            "HQ": "QG",
            "Industry": "secteur",
            "Total Funding": "capital",
            "ARR": "ca",
            "Valuation": "valorisation",
            "Employees": "nombre_employes",
            "Top Investors": "top_investisseurs",
            "Product": "produit",
            "G2 Rating": "cote_g2",
        }

df = df.rename(columns= renamed_columns)

In [648]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   nom_entreprise     100 non-null    object 
 1   annee_creation     100 non-null    int64  
 2   QG                 100 non-null    object 
 3   secteur            100 non-null    object 
 4   capital            99 non-null     object 
 5   ca                 100 non-null    object 
 6   valorisation       100 non-null    object 
 7   nombre_employes    100 non-null    object 
 8   top_investisseurs  100 non-null    object 
 9   produit            100 non-null    object 
 10  cote_g2            100 non-null    float64
dtypes: float64(1), int64(1), object(9)
memory usage: 8.7+ KB


Pas d'info sur le capital de SAP. Supprimer pour le moment.

In [649]:
df = df.dropna()

In [650]:
df["capital"].unique()

array(['$1B', '$65.4M', '$2.5M', '$2K', '$273M', '$82.5M', '$249.9M',
       '$145.5M', '$122.3M', '$60M', '$1.4B', '$100.5M', '$514.3M',
       '$353M', '$147.9M', '$311.2M', '$230.5M', '$261.3M', '$8.7B',
       '$590.5M', '$2.6B', '$572.6M', '$234.1M', '$453.2M', '$735.1M',
       '$332.9M', '$435.6M', '$85.5M', '$562.4M', '$1.7B', '$484.7M',
       '$778.2M', '$2B', '$3.5B', '$455.9M', '$349.5M', '$162M',
       '$173.7M', '$215M', '$40M', '$332.1M', '$81M', '$0', '$476.3M',
       '$400M', '$350.6M', '$240.7M', '$284M', '$330M', '$281.5M',
       '$336M', '$77M', '$235M', '$148M', '$481M', '$66.3M', '$44M',
       '$242M', '$7M', '$169.5M', '$402.7M', '$241.7M', '$120.8M', '$59M',
       '$902M', '$584M', '$489M', '$245M', '$175M', '$516M', '$1.2B',
       '$1.1B', '$734M', '$528M', '$1.5B', '$425M', '$347M', '$334M',
       '$930M', '$460M', '$228.5M', '$920M', '$750M', '$315M', '$899M',
       '$987M', '$450M'], dtype=object)

## Transformations

In [653]:
def tranform_column_capital(colonne: np.ndarray) -> np.ndarray:
    transformed_values = []
    
    for v in colonne:
        v = str(v)  
        v = v.lstrip("$").strip() 
        
        num_part = ""
        unit = None
        
        for c in v:
            if c.isdigit(): 
                num_part += c
            else:
                unit = c
                break

       
        if not num_part:  
            # num_part = np.nan
            print("Cette valeur n'est pas convertie")
        else:
            try:
                num_part = float(num_part)  # Conversion sécurisée
                if unit == "B":
                    num_part *= 1_000_000_000
                elif unit == "M":
                    num_part *= 1_000_000
            except ValueError:
                num_part = np.nan  # Gestion des erreurs
        
        transformed_values.append(num_part)

    return np.array(transformed_values)


def tranform_column_capital(colonne:np.ndarray) -> np.ndarray:
    transformed_values = []
    
    for v in colonne:
        v=str(v)
        v = v.lstrip("$")  
        num_part = ""
        unit = None 
        
        for c in v:
            if c.isdigit():
                num_part += c
            else:
                unit = c
                break
            
        try:
            num_part = int(num_part)
            if unit == "B":
                num_part *= 1_000_000_000
            elif unit == "M":
                num_part *= 1_000_000
        except ValueError:
            #num_part = np.nan
            print("Une des valeur contient des charactères non numériques")
            raise
        
        transformed_values.append(num_part)

    return np.array(transformed_values)

def transform_column_ca(colonne: np.ndarray) -> np.ndarray:
    transformed_ca = []
    
    for v in colonne:
        v=str(v)
        v = v.lstrip("$")  
        
        if v.endswith("T"):
            v = float(v[:-1]) * 1_000_000_000_000
        elif v.endswith("B"):
            v = float(v[:-1]) * 1_000_000_000
        elif v.endswith("M"):
            v = float(v[:-1]) * 1_000_000
        else:
            try:
                v = float(v)  
            except ValueError:
                # v = np.nan
                print("Une des valeur contient des charactères non numériques")
                raise
        
        transformed_ca.append(v)
    
    return np.array(transformed_ca)

def transform_column_valorisation(colonne: np.ndarray) -> np.ndarray:
    transformed_values = []
    
    for v in colonne:
        v=str(v)
        v = v.lstrip("$").split(" (")[0]
        
        unit = None
        if "T" in v:
            index_T = v.index("T")
            num_part = v[:index_T]
            unit = "T"
        elif "B" in v:
            index_B = v.index("B")
            num_part = v[:index_B]
            unit = "B"
        elif "M" in v:
            index_M = v.index("M")
            num_part = v[:index_M]
            unit = "M"
        else:
            num_part = v  
        
        try:
            num_part = float(num_part)
            if unit == "T":
                num_part *= 1_000_000_000_000
            elif unit == "B":
                num_part *= 1_000_000_000
            elif unit == "M":
                num_part *= 1_000_000
        except ValueError:
            # num_part = np.nan 
            print("Une des valeur contient des charactères non numériques")
            raise
        
        transformed_values.append(num_part)

    return np.array(transformed_values)

def tranform_column_nombre_employes(colonne: pd.Series) -> pd.Series:
    colonne = colonne.astype(str).apply(lambda v: v.replace(',', '')).astype(float)
    return colonne



In [654]:
df["capital_$"] = tranform_column_capital(df["capital"].to_numpy())
df["ca_$"] = transform_column_ca(df["ca"].to_numpy())
df["valorisation_$"] = transform_column_valorisation(df["valorisation"].to_numpy())
df["nombre_employes"] = tranform_column_nombre_employes(df["nombre_employes"])


df.drop(columns=["capital"], inplace=True)
df.drop(columns=["ca"], inplace=True)
df.drop(columns=["valorisation"], inplace=True)

In [655]:
df

,nom_entreprise,annee_creation,QG,secteur,nombre_employes,top_investisseurs,produit,cote_g2,capital_$,ca_$,valorisation_$
0,Microsoft,1975,"Redmond, WA, USA",Enterprise Software,221000.0,"Bill Gates, Paul Allen","Azure, Office 365, Teams",4.4,1000000000,2.700000e+11,3.000000e+12
1,Salesforce,1999,"San Francisco, CA, USA",CRM,75000.0,"Halsey Minor, Larry Ellison","Sales Cloud, Service Cloud",4.3,65,3.790000e+10,2.278000e+11
2,Adobe,1982,"San Jose, CA, USA",Creative Software,29945.0,Hambrecht & Quist,"Creative Cloud, Document Cloud",4.5,2,1.940000e+10,2.400000e+11
3,Oracle,1977,"Austin, TX, USA",Database & Enterprise,143000.0,"Larry Ellison, Bob Miner","Oracle Cloud, NetSuite",4.0,2,5.290000e+10,3.500000e+11
5,Intuit,1983,"Mountain View, CA, USA",Financial Software,18200.0,"Sierra Ventures, Kleiner Perkins","QuickBooks, TurboTax",4.4,273000000,1.440000e+10,1.800000e+11
...,...,...,...,...,...,...,...,...,...,...,...
95,CircleCI,2011,"San Francisco, CA, USA",CI/CD,500.0,"IVP, DFJ Growth",CI/CD Platform,4.4,315000000,1.000000e+08,1.700000e+09
96,ServiceTitan,2012,"Glendale, CA, USA",Field Service,2700.0,"Tiger Global, Sequoia",Field Service Management,4.2,1,6.850000e+08,9.500000e+09
97,Procore,2002,"Carpinteria, CA, USA",Construction,3200.0,"Bessemer, Tiger Global",Construction Management Platform,4.5,899000000,9.430000e+08,9.000000e+09
98,Automattic,2005,"San Francisco, CA, USA",Web Publishing,2000.0,"Salesforce Ventures, Insight","WordPress.com, WooCommerce",4.4,987000000,4.000000e+08,7.500000e+09
